# 02 — Serving & Tests (inférence + API)

Objectif : valider le **serving** du modèle (inférence) et préparer un **payload JSON** exploitable par l'API FastAPI.

Ce notebook couvre :
- Chargement du **model champion** + métadonnées (seuil)
- Prédiction locale (1 client) + scoring batch (optionnel)
- Construction d'un payload **JSON-sérialisable**
- Test de l'API déployée (optionnel, nécessite réseau)
- Génération d'une **fixture** de test (`tests/fixtures/sample_payload.json`)


### Sommaire
- [1. Préparation](#1-preparation)
- [2. Charger le modèle champion](#2-champion)
- [3. Inférence locale](#3-inference)
- [4. Scoring batch](#4-batch)
- [5. Payload JSON pour l'API](#5-payload)
- [6. Tester l'API FastAPI](#6-api)
- [7. Fixture pour tests](#7-fixture)


## <a id="1-preparation"></a>1. Préparation


### 1.1 Imports & configuration projet

Ce bloc :
- détecte la racine du projet
- ajoute le projet au `PYTHONPATH`
- importe les chemins (`RAW_DATA_DIR`, `MODELS_DIR`) et fonctions d'inférence


In [ ]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import json

PROJECT_ROOT = Path().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import RAW_DATA_DIR, MODELS_DIR
from src.inference import load_champion, predict_single


## <a id="2-champion"></a>2. Charger le modèle champion


Chargement du pipeline + lecture des métadonnées (dont le seuil).


In [ ]:
# Chargement du modèle champion et des métadonnées

champion = load_champion()
print(champion)

metadata_path = MODELS_DIR / "champion_metadata.json"
with open(metadata_path, "r") as f:
    metadata = json.load(f)

metadata


## <a id="3-inference"></a>3. Inférence locale


### 3.1 Charger un échantillon de données brutes


In [ ]:
# Chargement de quelques lignes du jeu d'entraînement brut

df_raw = pd.read_csv(RAW_DATA_DIR / "application_train.csv")
df_raw.shape

### 3.2 Construire un payload (1 client)

On extrait une ligne, on retire `TARGET`, puis on obtient un dictionnaire proche du JSON attendu par l'API.


In [ ]:
# Préparer un client exemple (on enlève TARGET)
sample_row = df_raw.drop(columns=["TARGET"]).iloc[0]

sample_payload = sample_row.to_dict()  # ce dict correspond à ton futur JSON pour l'API
list(sample_payload.items())[:10]  # aperçu des 10 premières features

### 3.3 Prédire pour un seul client


In [ ]:
# Prédiction pour un seul client avec le modèle champion

single_pred = predict_single(sample_payload, model=champion)
single_pred

## <a id="4-batch"></a>4. Scoring batch (optionnel)


Scorer plusieurs clients pour obtenir une vue d'ensemble rapide des prédictions.


In [ ]:
# Fonction utilitaire pour scorer plusieurs clients

def score_many(df, model=None, n_rows=1000, random_state=42):
    if model is None:
        model = load_champion()
    df_sample = df.sample(n_rows, random_state=random_state)

    X = df_sample.drop(columns=["TARGET"])
    y_true = df_sample["TARGET"].values

    proba = model.predict_proba(X)
    y_pred = model.predict(X)

    return df_sample.assign(
        proba_bad=proba,
        y_pred=y_pred,
    )

results = score_many(df_raw, model=champion, n_rows=500)
results.head()

In [ ]:
# Petite vue d'ensemble des prédictions

results[["TARGET", "y_pred"]].value_counts(normalize=True)

## <a id="5-payload"></a>5. Payload JSON pour l'API


### 5.1 Nettoyer le payload (types NumPy, NaN, inf)

L'API reçoit du JSON : on convertit donc les types NumPy en types Python et on remplace les valeurs non sérialisables (`NaN`, `inf`).


In [ ]:

def to_jsonable(val):
    # NaN ou valeurs manquantes -> None
    if pd.isna(val):
        return None

    # Infinis -> None (ou un autre traitement si tu préfères)
    if isinstance(val, (np.floating, float)) and np.isinf(val):
        return None

    # Numpy int -> int Python
    if isinstance(val, (np.integer,)):
        return int(val)

    # Numpy float -> float Python
    if isinstance(val, (np.floating,)):
        return float(val)

    # Tout le reste (str, bool, etc.) on laisse tel quel
    return val


# On repart d'une ligne brute du df
sample_row = df_raw.drop(columns=["TARGET"]).iloc[0]

clean_payload = {
    col: to_jsonable(val)
    for col, val in sample_row.items()
}

# Vérifier que ça passe bien en JSON
print(json.dumps(clean_payload))  # affiche tout le JSON

### 5.2 Exemple JSON (tronqué)


In [ ]:
# Exemple de payload JSON (propre) à utiliser pour tester l'API FastAPI
example_payload = json.dumps(clean_payload, indent=2, ensure_ascii=False)
print(example_payload[:1000])  # on tronque l'affichage si c'est trop long


## <a id="6-api"></a>6. Tester l'API FastAPI (optionnel)


Ces appels nécessitent un accès réseau et une API disponible.

- Pour tester en local, remplacer `base_url` par `http://127.0.0.1:8000` .


In [ ]:
import requests

base_url = "https://home-credit-project.onrender.com"

# 1) Wake up + healthcheck
health_url = f"{base_url}/health"
requests.get(health_url, timeout=30).json()

In [ ]:
predict_url = f"{base_url}/predict"
response = requests.post(predict_url, json=clean_payload, timeout=60)
response.status_code, response.json()

## <a id="7-fixture"></a>7. Fixture pour tests


On sauvegarde le `clean_payload` au format JSON dans `tests/fixtures/` afin de le réutiliser dans des tests automatisés.


In [ ]:
fixtures_dir = PROJECT_ROOT / "tests" / "fixtures"
fixtures_dir.mkdir(parents=True, exist_ok=True)

fixture_path = fixtures_dir / "sample_payload.json"

with open(fixture_path, "w", encoding="utf-8") as f:
    json.dump(clean_payload, f, ensure_ascii=False, indent=2)

print("✅ Fixture saved:", fixture_path)
print("Keys:", len(clean_payload))

## Conclusion

Le notebook fournit un exemple de payload réutilisable pour le serving (API) et une fixture pour les tests.
